# Ex06 - Introdução ao OpenGL

Nesta atividade, vocês vão exercitar os primeiros conceitos de OpenGL. Para evitar erros de execução, utilize apenas uma célula de código para cada parte desta atividade.

### Parte 1 - Renderização de objetos simples

Crie um programa que reproduza uma imagem semelhante à imagem abaixo (triângulos nos cantos e um quadrado no centro). O tamanho e cor dos objetos e a cor do background é livre, mas as arestas dos triângulos devem estar coladas nos cantos da janela. Você pode usar como base os notebooks notebook ([17_Primeiro_programa_OpenGL](17_Primeiro_programa_OpenGL.ipynb)), ([18_Configurando_uma_cor_unica_para_todos_os_vertices](18_Configurando_uma_cor_unica_para_todos_os_vertices.ipynb)) e ([19_Renderizando_mais_de_um_objeto](19_Renderizando_mais_de_um_objeto.ipynb)).

![title](cg/images/ex6_image.png)

Here, we base our selves on the tutorial codes. What we change is the position of the two original triangles (put them in the corners), and create two new triangles which together create a centered rectangle. We also change the background color to white, and set colors so that each shape has a different color.

In [4]:
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # each triangles' vertex positions
        vertex_position_01 = np.array([ # Triangle 1
            -1.00, -1.00, 0.0, 1.0, 
            -0.60, -1.00, 0.0, 1.0,
            -1.00, -0.60, 0.0, 1.0],
            dtype = np.float32)
        vertex_position_02 = np.array([ # Triangle 2
             1.00,  1.00, 0.0, 1.0, 
             0.60,  1.00, 0.0, 1.0,
             1.00,  0.60, 0.0, 1.0],
            dtype=np.float32)
        vertex_position_03 = np.array([ # Triangle 3
            -0.20,  0.20, 0.0, 1.0,
             0.20,  0.20, 0.0, 1.0,
            -0.20, -0.20, 0.0, 1.0],
            dtype = np.float32)
        vertex_position_04 = np.array([ # Triangle 4
             0.20, -0.20, 0.0, 1.0,
             0.20,  0.20, 0.0, 1.0,
            -0.20, -0.20, 0.0, 1.0],
            dtype = np.float32)
        
        # create objects responsable for loading data on to the GPU and rendering it
        self.triangleRenderer01 = ModelRenderer(vertex_position_01)
        self.triangleRenderer02 = ModelRenderer(vertex_position_02)
        self.triangleRenderer03 = ModelRenderer(vertex_position_03)
        self.triangleRenderer04 = ModelRenderer(vertex_position_04)
        
        # create simple Shader Program
        self.shaderProgram = SimpleShaderProgram()
        
        # activate the shader program to allow us to configure a single color for all vertices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # store pointers to shader program input variables
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configure the model data to be shader inputs
        self.triangleRenderer01.setVertexPositionLoc(position_loc)
        self.triangleRenderer02.setVertexPositionLoc(position_loc)
        self.triangleRenderer03.setVertexPositionLoc(position_loc)
        self.triangleRenderer04.setVertexPositionLoc(position_loc)
    
    
    def paintGL(self):
        
        # configure background color (white)
        gl.glClearColor(1, 1, 1, 1)
        
        # clean background with the specified color
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # activate the shader program (to be executed by GPU)
        self.shaderProgram.bind()
        
        # configure 1st triangle's color and render
        self.shaderProgram.setUniformColor(np.array([0.0, 0.0, 0.5, 1.0], dtype=np.float32))
        self.triangleRenderer01.render()
        
        # configure 2nd triangle's color and render
        self.shaderProgram.setUniformColor(np.array([0.5, 0.0, 0.0, 1.0], dtype=np.float32))
        self.triangleRenderer02.render()
        
        # configure 3rd triangle's color and render
        self.shaderProgram.setUniformColor(np.array([0.0, 0.5, 0.0, 1.0], dtype=np.float32))
        self.triangleRenderer03.render()
        
        # configure 4th triangle's color and render
        self.shaderProgram.setUniformColor(np.array([0.0, 0.5, 0.0, 1.0], dtype=np.float32))
        self.triangleRenderer04.render()
        
        # deactivate shader program
        self.shaderProgram.release()
        
        # re-call paintGL to update values
        self.update()

    def resizeGL(self, width, height):
        
        # update rendering area to be the entire window
        gl.glViewport(0, 0, width, height)

def main():
    import sys

    # create Qt application
    app = QApplication(sys.argv)

    # Specify the OpenGL context
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    # create rendering window
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('Exercise 1')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

Here is what we view in our rendering window:

![4triangles.PNG](4triangles.PNG)

### Parte 2 - Animação

Crie um programa que tenha um objeto (triângulo, quadrado, etc) desenhado no centro, e que as cores deste objeto e do background mudam ao longo do tempo. Você pode usar como base o notebook ([20_Animando_as_cores_dos_triangulos](20_Animando_as_cores_dos_triangulos.ipynb))

Here, we base our selves on the above code. We delete the corner triangles and keep the center rectangle (two triangles). We then base ourselves on the tutorial #20 code to make colors change with time by interpoling two color arrays. Instead of applying the two colors to the two triangles though, we apply one to the background and one to both triangles (the rectangle).

In [1]:
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    
    def initializeGL(self):
        
        # each triangles' vertex positions
        vertex_position_01 = np.array([
            -0.50, -0.50, 0.0, 1.0, # Triangle 1
             0.50, -0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0],
            dtype=np.float32)
        
        vertex_position_02 = np.array([
             0.50, -0.50, 0.0, 1.0, # Triangle 2
             0.50,  0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0],
            dtype=np.float32)
        
        # create objects responsable for loading data on to the GPU and rendering it
        self.triangleRenderer01 = ModelRenderer(vertex_position_01)
        self.triangleRenderer02 = ModelRenderer(vertex_position_02)
        
        # create simple Shader Program
        self.shaderProgram = SimpleShaderProgram()
        
        # activate the shader program to allow us to configure a single color for all vertices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # store pointers to shader program input variables
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configure the model data to be shader inputs
        self.triangleRenderer01.setVertexPositionLoc(position_loc)
        self.triangleRenderer02.setVertexPositionLoc(position_loc)

        # store program start time
        self.startTime = time.time()
        
    def paintGL(self):
        
        # calculate the current program time
        self.currentTime = time.time()
        time_difference = self.currentTime - self.startTime
        
        # calculate the interpolation factor
        interpolation_factor = (math.sin(time_difference) + 1) / 2
        
        color_01 = np.array([0.0, 1.0, 0.0, 1.0], dtype=np.float32) #green
        color_02 = np.array([0.0, 0.0, 0.0, 1.0], dtype=np.float32) #black
        
        #calculate both color arrays
        mixed_color_01 = interpolation_factor * color_01 + (1 - interpolation_factor) * color_02
        mixed_color_02 = interpolation_factor * color_02 + (1 - interpolation_factor) * color_01
        
        # activate the shader program (run by GPU)
        self.shaderProgram.bind()
        
        # set background color
        gl.glClearColor(mixed_color_02[0],mixed_color_02[1],mixed_color_02[2],mixed_color_02[3])
        
        # clean background with the specified color
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # configure 1st triangle's color and render
        self.shaderProgram.setUniformColor(mixed_color_01)
        self.triangleRenderer01.render()
        
        # configure 2nd triangle's color and render
        self.shaderProgram.setUniformColor(mixed_color_01)
        self.triangleRenderer02.render()
        
        # deactivate the shader program
        self.shaderProgram.release()
        
        # re-call paintGL to update values
        self.update()

    def resizeGL(self, width, height):
        
        # update rendering area to be the entire window
        gl.glViewport(0, 0, width, height)

def main():
    import sys

    # create Qt application
    app = QApplication(sys.argv)

    # specify the OpenGL context
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    # create rendering window
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Andre\Anaconda2\envs\EA979\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Here is what we view in our rendering window:

![changing.gif](changing.gif)

### Parte 3 - Evento de teclado

Crie um programa que tenha um objeto (triângulo, quadrado, etc) desenhado e utilize a setas do teclado para deslocar este objeto na janela. O objeto deve se deslocar na direção da teclada pressionada. Você pode usar como base o notebook ([24_Eventos_do_teclado.ipynb](24_Eventos_do_teclado.ipynb))

Here we base ourselves on tutorial #24 to add keyboard events. But instead of changing color as is done in the tutorial, we update the objects' position. We also add the functionality of using the AWSD keys as arrows instead of the arrows as many games have (can still use arrows if wanted).

In [7]:
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # each triangles' vertex positions
        vertex_position_01 = np.array([
            -0.25, -0.25, 0.0, 1.0, # Triangle 1
             0.25, -0.25, 0.0, 1.0,
            -0.25,  0.25, 0.0, 1.0],
            dtype=np.float32)
        
        vertex_position_02 = np.array([
             0.25, -0.25, 0.0, 1.0, # Triangle 2
             0.25,  0.25, 0.0, 1.0,
            -0.25,  0.25, 0.0, 1.0],
            dtype=np.float32)
        
        # save array in self.position01 attribute
        self.position01 = vertex_position_01
        
        # save array in self.position02 attribute
        self.position02 = vertex_position_02
        
        # create objects responsable for loading data on to the GPU and rendering it
        self.triangleRenderer01 = ModelRenderer(self.position01)
        self.triangleRenderer02 = ModelRenderer(self.position02)
        
        # create simple Shader Program
        self.shaderProgram = SimpleShaderProgram()
        
        # activate the shader program to allow us to configure a single color for all vertices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # store pointers to shader program input variables
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configure the model data to be shader inputs
        self.triangleRenderer01.setVertexPositionLoc(position_loc)
        self.triangleRenderer02.setVertexPositionLoc(position_loc)

        # set self.color atributes to the same color (torquoise)
        self.colorTriangle01 = np.array([0.0, 0.5, 0.5, 1.0], dtype=np.float32)
        self.colorTriangle02 = np.array([0.0, 0.5, 0.5, 1.0], dtype=np.float32)
        
    def paintGL(self):
        
        # set background color
        gl.glClearColor(0, 0, 0, 1)
        
        # clean background with the specified color
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # activate the shader program (run by GPU)
        self.shaderProgram.bind()
        
        # set 1st triangle's color and render
        self.shaderProgram.setUniformColor(self.colorTriangle01)
        self.triangleRenderer01.render()
        
        # set 2nd triangle's color and render
        self.shaderProgram.setUniformColor(self.colorTriangle02)
        self.triangleRenderer02.render()
        
        # deactivate the Shader Program
        self.shaderProgram.release()
        
        # re-call paintGL to update values
        self.update()

    def resizeGL(self, width, height):
        
        # update rendering area to be entire window
        gl.glViewport(0, 0, width, height)
        
    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        # step
        step = 0.2
        
        # if "up arrow" or "W" is pressed, move rectangle up
        if ((event.key() == (QtCore.Qt.Key_Up)) or (event.key() == (QtCore.Qt.Key_W))):
            self.updatePosition(0.2,'up')

        # if "down arrow" or "S" is pressed, move rectangle down
        elif ((event.key() == (QtCore.Qt.Key_Down)) or (event.key() == (QtCore.Qt.Key_S))):
            self.updatePosition(0.2,'down')
        
        # if "left arrow" or "A" is pressed, move rectangle to the left
        elif ((event.key() == (QtCore.Qt.Key_Left)) or (event.key() == (QtCore.Qt.Key_A))):
            self.updatePosition(0.2,'left')
        
        # if "right arrow" or "D" is pressed, move rectangle to the right
        elif ((event.key() == (QtCore.Qt.Key_Right)) or (event.key() == (QtCore.Qt.Key_D))):
            self.updatePosition(0.2,'right')

            
    def updatePosition(self,step,direction):

        if direction == 'up':
            self.position01 += [0,step,0,0,
                                0,step,0,0,
                                0,step,0,0]
            self.position02 += [0,step,0,0,
                                0,step,0,0,
                                0,step,0,0]
            self.triangleRenderer01 = ModelRenderer(self.position01)
            self.triangleRenderer02 = ModelRenderer(self.position02)
            position_loc = self.shaderProgram.getVertexPositionLoc()
            self.triangleRenderer01.setVertexPositionLoc(position_loc)
            self.triangleRenderer02.setVertexPositionLoc(position_loc)

        if direction == 'down':
            self.position01 += [0,-step,0,0,
                                0,-step,0,0,
                                0,-step,0,0]
            self.position02 += [0,-step,0,0,
                                0,-step,0,0,
                                0,-step,0,0]
            self.triangleRenderer01 = ModelRenderer(self.position01)
            self.triangleRenderer02 = ModelRenderer(self.position02)
            position_loc = self.shaderProgram.getVertexPositionLoc()
            self.triangleRenderer01.setVertexPositionLoc(position_loc)
            self.triangleRenderer02.setVertexPositionLoc(position_loc)

        if direction == 'left':
            self.position01 += [-step,0,0,0,
                                -step,0,0,0,
                                -step,0,0,0]
            self.position02 += [-step,0,0,0,
                                -step,0,0,0,
                                -step,0,0,0]
            self.triangleRenderer01 = ModelRenderer(self.position01)
            self.triangleRenderer02 = ModelRenderer(self.position02)
            position_loc = self.shaderProgram.getVertexPositionLoc()
            self.triangleRenderer01.setVertexPositionLoc(position_loc)
            self.triangleRenderer02.setVertexPositionLoc(position_loc)

        if direction == 'right':
            self.position01 += [step,0,0,0,
                                step,0,0,0,
                                step,0,0,0]
            self.position02 += [step,0,0,0,
                                step,0,0,0,
                                step,0,0,0]
            self.triangleRenderer01 = ModelRenderer(self.position01)
            self.triangleRenderer02 = ModelRenderer(self.position02)
            position_loc = self.shaderProgram.getVertexPositionLoc()
            self.triangleRenderer01.setVertexPositionLoc(position_loc)
            self.triangleRenderer02.setVertexPositionLoc(position_loc)
                    
def main():
    import sys

    # create Qt application
    app = QApplication(sys.argv)

    # specify the OpenGL context
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    # create rendering window
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

Here we have our rendering window responding to key press actions:

![keys.gif](keys.gif)